In [3]:
import pandas as pd
import numpy as np

def wrangle(filename):
    """
    Wrangle function that takes the name of a CSV file as input and returns a DataFrame.
    
    Parameters:
    filename (str): Path to the CSV file
    
    Returns:
    pandas.DataFrame: Cleaned and processed DataFrame
    """
    
    # Read the CSV file
    df = pd.read_csv(filename)
    
    # 1. Subset data: Only apartments in Mexico City ("Distrito Federal") under $100,000
    df = df[
        (df['property_type'] == 'apartment') & 
        (df['place_with_parent_names'].str.contains('|Distrito Federal|', na=False)) &
        (df['price_aprox_usd'] < 100000)
    ].copy()
    
    # 2. Remove outliers: Trim bottom and top 10% by surface_covered_in_m2
    # Remove rows where surface_covered_in_m2 is null first
    df = df.dropna(subset=['surface_covered_in_m2'])
    
    # Calculate 10th and 90th percentiles
    q10 = df['surface_covered_in_m2'].quantile(0.1)
    q90 = df['surface_covered_in_m2'].quantile(0.9)
    
    # Filter to keep only rows between 10th and 90th percentiles
    df = df[(df['surface_covered_in_m2'] >= q10) & (df['surface_covered_in_m2'] <= q90)]
    
    # 3. Create separate "lat" and "lon" columns
    # Extract coordinates from lat-lon column
    def extract_coordinates(lat_lon_str):
        if pd.isna(lat_lon_str) or lat_lon_str == '':
            return pd.Series([np.nan, np.nan])
        try:
            # Remove quotes and split by comma
            coords = str(lat_lon_str).strip('"').split(',')
            if len(coords) == 2:
                lat = float(coords[0])
                lon = float(coords[1])
                return pd.Series([lat, lon])
            else:
                return pd.Series([np.nan, np.nan])
        except:
            return pd.Series([np.nan, np.nan])
    
    df[['lat', 'lon']] = df['lat-lon'].apply(extract_coordinates)
    
    # 4. Create "borough" feature from "place_with_parent_names" column
    def extract_borough(place_str):
        if pd.isna(place_str):
            return np.nan
        try:
            # Split by '|' and get the first non-empty part (borough)
            parts = str(place_str).split('|')
            for part in parts:
                if part.strip() and part.strip() != '':
                    return part.strip()
            return np.nan
        except:
            return np.nan
    
    df['borough'] = df['place_with_parent_names'].apply(extract_borough)
    
    # 5. Drop columns that are more than 50% null values
    null_percentage = df.isnull().sum() / len(df)
    cols_to_keep = null_percentage[null_percentage <= 0.5].index.tolist()
    df = df[cols_to_keep]
    
    # 6. Drop columns containing low- or high-cardinality categorical values
    categorical_cols = df.select_dtypes(include=['object']).columns
    
    cols_to_drop_cardinality = []
    for col in categorical_cols:
        unique_count = df[col].nunique()
        total_count = len(df)
        
        # Drop if very low cardinality (< 2 unique values) or very high cardinality (> 50% unique)
        if unique_count < 2 or (unique_count / total_count) > 0.5:
            cols_to_drop_cardinality.append(col)
    
    df = df.drop(columns=cols_to_drop_cardinality, errors='ignore')
    
    # 7. Drop columns that would constitute leakage for target "price_aprox_usd"
    leakage_cols = [
        'price',  # Original price in local currency
        'price_aprox_local_currency',  # Price in local currency
        'price_usd_per_m2',  # Price per m2 in USD (derived from target)
        'price_per_m2',  # Price per m2 in local currency (derived from target)
    ]
    
    df = df.drop(columns=[col for col in leakage_cols if col in df.columns], errors='ignore')
    
    # 8. Drop columns that would create multicollinearity issues
    multicollinear_cols = [
        'surface_total_in_m2',  # Highly correlated with surface_covered_in_m2
        'lat-lon',  # Redundant now that we have lat and lon separately
        'place_with_parent_names',  # Redundant now that we have borough
        'properati_url',  # Not useful for modeling
        'currency',  # Should be consistent after filtering, and not useful for modeling
        'operation',  # Should be consistent (all 'sell') after filtering
    ]
    
    df = df.drop(columns=[col for col in multicollinear_cols if col in df.columns], errors='ignore')
    
    # Reset index
    df = df.reset_index(drop=True)
    
    return df


In [4]:
# Test the wrangle function with Mexico City data
df = wrangle('data/mexico/mexico-city-real-estate-1.csv')

print("Shape of processed data:", df.shape)
print("\nColumns in processed data:")
print(df.columns.tolist())
print("\nFirst few rows:")
print(df.head())
print("\nData types:")
print(df.dtypes)
print("\nNull values per column:")
print(df.isnull().sum())
print("\nUnique values in borough column:")
print(df['borough'].value_counts().head(10))


Shape of processed data: (1132, 6)

Columns in processed data:
['Unnamed: 0', 'price_aprox_usd', 'surface_covered_in_m2', 'lat', 'lon', 'borough']

First few rows:
   Unnamed: 0  price_aprox_usd  surface_covered_in_m2        lat         lon  \
0          12         94022.66                   57.0  23.634501 -102.552788   
1          21         70880.12                   56.0  19.402413  -99.095391   
2          22         68228.99                   80.0  19.357820  -99.149406   
3          23         24235.78                   60.0  19.504985  -99.208557   
4          27         94140.20                   50.0  19.354219  -99.126244   

         borough  
0  Benito Juárez  
1      Iztacalco  
2  Benito Juárez  
3   Azcapotzalco  
4       Coyoacán  

Data types:
Unnamed: 0                 int64
price_aprox_usd          float64
surface_covered_in_m2    float64
lat                      float64
lon                      float64
borough                   object
dtype: object

Null values per

In [10]:
frame1 = wrangle("data/mexico/mexico-city-real-estate-1.csv")
frame1.isnull().sum() / len(frame1)

Unnamed: 0               0.000000
price_aprox_usd          0.000000
surface_covered_in_m2    0.000000
lat                      0.055654
lon                      0.055654
borough                  0.000000
dtype: float64

In [12]:
df.select_dtypes("object").head()


,borough
0,Benito Juárez
1,Iztacalco
2,Benito Juárez
3,Azcapotzalco
4,Coyoacán


In [13]:
df.select_dtypes("object").nunique()

borough    20
dtype: int64

In [20]:
# Use this cell to test your wrangle function on the file `mexico-city-real-estate-1.csv`
frame1 = wrangle("data/mexico/mexico-city-real-estate-1.csv")

frame1=frame1.drop(range(1101,1132), axis="index")

frame1.shape

(1101, 6)